In [ ]:
def read_file(filepath):
  with open(filepath) as f:
    str_text = f.read()

  return str_text

In [ ]:
# print(read_file('/content/drive/MyDrive/Data Science/Text Generation with LSTM/moby_dick_four_chapters.txt'))

In [ ]:
# import spacy.cli
# spacy.cli.download('en_core_web_lg')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'ner'])

In [ ]:
nlp.max_length = 1198623

In [ ]:
def separate_punc(doc_text):
  return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [ ]:
d = read_file('/content/drive/MyDrive/Data Science/Text Generation with LSTM/moby_dick_four_chapters.txt')

In [ ]:
tokens = separate_punc(d)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
import numpy as np

# ***Error***

In [ ]:
# Creating a Data Structure with 24 timesteps and 1 output
X_train = []
for i in range(26, len(tokens)):
  X_train.append(tokens[i-26:i])

In [ ]:
type(X_train)

list

In [ ]:
len(X_train[1])

26

In [ ]:
!pip install Keras-Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
print(X_train[0])

[956, 14, 263, 51, 261, 408, 87, 219, 129, 111, 954, 260, 50, 43, 38, 314, 7, 23, 546, 3, 150, 259, 6, 2713, 14, 24]


In [ ]:
# for i in X_train[0]:
#   print(f"{i}: {tokenizer.index_word[i]}")

In [ ]:
vocab_size = len(tokenizer.word_counts)

In [ ]:
vocab_size

2718

In [ ]:
X_train = np.array(X_train)

In [ ]:
X_train

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [ ]:
X_train.shape

(11312, 26)

In [ ]:
from keras.utils import to_categorical

In [ ]:
X = X_train[:,:-1]
y = X_train[:,-1]

In [ ]:
y = to_categorical(y, num_classes=vocab_size+1)

In [ ]:
seq_len = X.shape[1]

In [ ]:
X.shape

(11312, 25)

In [ ]:
y.shape

(11312, 2719)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense, Input

In [ ]:
model = Sequential()

# Directly use LSTM layers without an Embedding layer
model.add(Input(shape=(seq_len,)))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=32))
model.add(Dropout(rate=0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.1))

model.add(Dense(2718, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

ValueError: Input 0 of layer "lstm_6" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 25)

In [ ]:
model.fit(X,y, batch_size=128, epochs=5, verbose=1)

Epoch 1/5


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_3_1/Cast:0", shape=(None, 25), dtype=float32). Expected shape (None, 2718, 25), but input has incompatible shape (None, 25)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 25), dtype=int64)
  • training=True
  • mask=None

# ***Solution***

In [ ]:
# Creating a Data Structure with 24 timesteps and 1 output
X_train = []
for i in range(26, len(tokens)):
  X_train.append(tokens[i-26:i])

In [ ]:
type(X_train)

list

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
vocab_size = len(tokenizer.word_counts)
vocab_size

2718

In [ ]:
X_train = np.array(X_train)

In [ ]:
X_train[1]

array([  14,  263,   51,  261,  408,   87,  219,  129,  111,  954,  260,
         50,   43,   38,  314,    7,   23,  546,    3,  150,  259,    6,
       2713,   14,   24,  957])

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
from keras.utils import to_categorical

In [ ]:
X = X_train[:,:-1]
y = X_train[:,-1]

In [ ]:
y = to_categorical(y, num_classes=vocab_size+1)

In [ ]:
seq_len = X.shape[1]

In [ ]:
X.shape, y.shape

((11312, 25), (11312, 2719))

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size+1, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))

model.add(Dense(vocab_size+1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X,y,batch_size=128, epochs=5, verbose=1)

Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 71ms/step - accuracy: 0.0308 - loss: 7.4624
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.0562 - loss: 6.3437
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.0491 - loss: 6.3229
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.0529 - loss: 6.2177
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - accuracy: 0.0516 - loss: 6.1564


# ***Practice***

In [ ]:
print(tokens)

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i', 'thought', 'i', 'would', 'sail', 'about', 'a', 'little', 'and', 'see', 'the', 'watery', 'part', 'of', 'the', 'world', 'it', 'is', 'a', 'way', 'i', 'have', 'of', 'driving', 'off', 'the', 'spleen', 'and', 'regulating', 'the', 'circulation', 'whenever', 'i', 'find', 'myself', 'growing', 'grim', 'about', 'the', 'mouth', 'whenever', 'it', 'is', 'a', 'damp', 'drizzly', 'november', 'in', 'my', 'soul', 'whenever', 'i', 'find', 'myself', 'involuntarily', 'pausing', 'before', 'coffin', 'warehouses', 'and', 'bringing', 'up', 'the', 'rear', 'of', 'every', 'funeral', 'i', 'meet', 'and', 'especially', 'whenever', 'my', 'hypos', 'get', 'such', 'an', 'upper', 'hand', 'of', 'me', 'that', 'it', 'requires', 'a', 'strong', 'moral', 'principle', 'to', 'prevent', 'me', 'from', '

In [ ]:
tokens = np.array(tokens)

In [ ]:
tokens

array(['call', 'me', 'ishmael', ..., 'marshal', "'s", 'baton'],
      dtype='<U25')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)
tokens = tokenizer.texts_to_sequences(tokens)

In [ ]:
tokens[0]

[552]

In [ ]:
new_tokens = [token[0] for token in tokens if token]

In [ ]:
new_tokens

[552,
 14,
 222,
 49,
 223,
 319,
 84,
 197,
 121,
 104,
 553,
 224,
 50,
 40,
 36,
 264,
 7,
 22,
 412,
 3,
 136,
 225,
 6,
 958,
 14,
 24,
 959,
 5,
 60,
 5,
 57,
 320,
 38,
 2,
 50,
 3,
 88,
 1,
 960,
 166,
 4,
 1,
 167,
 8,
 20,
 2,
 105,
 5,
 47,
 4,
 961,
 96,
 1,
 962,
 3,
 963,
 1,
 964,
 265,
 5,
 413,
 97,
 965,
 966,
 38,
 1,
 266,
 265,
 8,
 20,
 2,
 414,
 967,
 968,
 7,
 22,
 415,
 265,
 5,
 413,
 97,
 554,
 555,
 112,
 267,
 556,
 3,
 969,
 31,
 1,
 970,
 4,
 198,
 971,
 5,
 972,
 3,
 321,
 265,
 22,
 973,
 113,
 68,
 48,
 974,
 130,
 4,
 14,
 9,
 8,
 557,
 2,
 416,
 975,
 976,
 6,
 977,
 14,
 28,
 978,
 979,
 39,
 1,
 322,
 3,
 980,
 981,
 558,
 25,
 982,
 96,
 46,
 5,
 417,
 8,
 268,
 74,
 6,
 113,
 6,
 72,
 16,
 168,
 16,
 5,
 122,
 18,
 20,
 22,
 983,
 23,
 984,
 3,
 985,
 15,
 2,
 986,
 987,
 988,
 989,
 123,
 51,
 11,
 418,
 5,
 419,
 151,
 6,
 1,
 199,
 29,
 20,
 136,
 990,
 7,
 18,
 54,
 55,
 13,
 269,
 8,
 137,
 21,
 270,
 7,
 106,
 420,
 49,
 74,
 40,
 76,
 991,

In [ ]:
training_set = np.array(new_tokens)
training_set

array([ 552,   14,  222, ..., 2697,   25, 2698])

In [ ]:
type(training_set)

numpy.ndarray

In [ ]:
training_set = training_set.reshape(-1, 1)
training_set.shape

(11338, 1)

In [ ]:
# Creating a Data Structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set)):
  X_train.append(training_set[i-60:i, 0])
  y_train.append(training_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape[0], X_train.shape[1]

(11278, 60)

In [ ]:
X_train.shape

(11278, 60)

In [ ]:
X_train.shape[1]

60

In [ ]:
seq_len = X_train.shape[1]

In [ ]:
X_train

array([[ 552,   14,  222, ...,  964,  265,    5],
       [  14,  222,   49, ...,  265,    5,  413],
       [ 222,   49,  223, ...,    5,  413,   97],
       ...,
       [  19,   18, 2689, ...,  243,   52,    2],
       [  18, 2689,   62, ...,   52,    2, 2697],
       [2689,   62,    5, ...,    2, 2697,   25]])

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train

array([[[ 552],
        [  14],
        [ 222],
        ...,
        [ 964],
        [ 265],
        [   5]],

       [[  14],
        [ 222],
        [  49],
        ...,
        [ 265],
        [   5],
        [ 413]],

       [[ 222],
        [  49],
        [ 223],
        ...,
        [   5],
        [ 413],
        [  97]],

       ...,

       [[  19],
        [  18],
        [2689],
        ...,
        [ 243],
        [  52],
        [   2]],

       [[  18],
        [2689],
        [  62],
        ...,
        [  52],
        [   2],
        [2697]],

       [[2689],
        [  62],
        [   5],
        ...,
        [   2],
        [2697],
        [  25]]])

In [ ]:
X_train.shape

(11278, 60, 1)

In [ ]:
y_train.shape

(11278,)

In [ ]:
vocabulary_size = len(tokenizer.word_counts) + 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input

In [ ]:
model = Sequential()

# Directly use LSTM layers without an Embedding layer
model.add(Input(shape=(X_train.shape[1], 1)))

model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=32))
model.add(Dropout(rate=0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.1))

model.add(Dense(vocabulary_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 60, 128)             │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 60, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2699)                │          89,067 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 218,507 (853.54 KB)

 Trainable params: 218,507 (853.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train_one_hot = to_categorical(y_train, num_classes=vocabulary_size)

In [ ]:
history = model.fit(X_train, y_train_one_hot, batch_size=128, epochs=50, verbose=1)

Epoch 1/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 32s 356ms/step - accuracy: 0.0631 - loss: 6.0499
Epoch 2/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 35s 391ms/step - accuracy: 0.0619 - loss: 6.0732
Epoch 3/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 32s 358ms/step - accuracy: 0.0643 - loss: 6.0001
Epoch 4/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 41s 359ms/step - accuracy: 0.0674 - loss: 6.0041
Epoch 5/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 40s 352ms/step - accuracy: 0.0655 - loss: 6.0145
Epoch 6/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 44s 383ms/step - accuracy: 0.0654 - loss: 5.9804
Epoch 7/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 38s 353ms/step - accuracy: 0.0618 - loss: 5.9731
Epoch 8/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 41s 357ms/step - accuracy: 0.0645 - loss: 5.9455
Epoch 9/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 40s 353ms/step - accuracy: 0.0692 - loss: 5.9031
Epoch 10/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 44s 385ms/step - accuracy: 0.0662 - loss: 5.9329
Epoch 11/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 39s 357ms/step - accuracy: 0.0667 - loss: 5.9089
Epoch 12/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 42

In [ ]:
from pickle import dump, load

In [ ]:
model.save('/content/drive/MyDrive/Data Science/Text Generation with LSTM/Text-Generation.keras')

In [ ]:
dump(tokenizer,open('Text-Generation-Tokenizer', 'wb'))

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
seed_text = 'and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking'

In [ ]:
seed_text2 = 'As the sun began to set behind the mountains, casting long shadows across the valley, the wind picked up, rustling the leaves in the ancient oak trees that lined the narrow path. The sound of distant thunder echoed through the hills, a storm brewing on the horizon, while the travelers hurried to find shelter before the rain began to pour down'

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text

    for _ in range(num_gen_words):
        # Encode the input text
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]

        # Pad the encoded text to the required sequence length
        if len(encoded_text) != 60:
          encoded_text = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        in_text = np.array(encoded_text)
        in_text = in_text.reshape(1, 60, 1)

        # Predict the next word's index
        pred_probs = model.predict(in_text, verbose=0)[0]

        # Get the index of the word with the highest probability
        pred_word_index = np.argmax(pred_probs)

        # Retrieve the corresponding word from the tokenizer's index_word dictionary
        pred_word = tokenizer.index_word.get(pred_word_index, '')

        # Append the predicted word to the input text
        input_text += ' ' + pred_word

        # Store the predicted word in the output text
        output_text.append(pred_word)

    return ' '.join(output_text)

In [ ]:
generate_text(model, tokenizer=tokenizer, seq_len=seq_len, seed_text=seed_text2, num_gen_words=20)

'the little and the room of the room of the room of the room of the room of the room'

In [ ]:
model.save('/content/drive/MyDrive/Data Science/Text Generation with LSTM/andtheroom.keras')

In [ ]:
dump(tokenizer,open('/content/drive/MyDrive/Data Science/Text Generation with LSTM/androomtokenizer', 'wb'))

## ***Testing***

In [ ]:
encoded_text = tokenizer.texts_to_sequences([seed_text])[0]
encoded_text = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

In [ ]:
print(encoded_text)

[[ 376    1 2435    6   30  140   10  342   71   18    7   27  119    2
  2436   13    2  342  446    3 2437  105    5  160  132]]


In [ ]:
input_text = np.array(encoded_text)

In [ ]:
input_text.shape

(1, 25)

In [ ]:
input_text = input_text.reshape(1, 25, 1)

In [ ]:
input_text.shape

(1, 25, 1)

In [ ]:
pred_probs = model.predict(input_text, verbose=0)[0]
print(pred_probs)

[9.00787089e-09 4.94035855e-02 2.85572633e-02 ... 2.13851649e-07
 1.38735405e-08 1.42094257e-06]


In [ ]:
# Get the index of the word with the highest probability
pred_word_index = np.argmax(pred_probs)

# Retrieve the corresponding word from the tokenizer's index_word dictionary
pred_word = tokenizer.index_word.get(pred_word_index, '')

# Debugging: Print the predicted word and its index
print(f"Predicted index: {pred_word_index}, Predicted word: '{pred_word}'")

Predicted index: 3, Predicted word: 'and'


# ***New Testing***

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
n_model = load_model('/content/drive/MyDrive/Data Science/Text Generation with LSTM/Text-Generation.h5')
# n_tokenizer = load(open('/content/drive/MyDrive/Data Science/Text Generation with LSTM/epochBIG', 'rb'))

In [ ]:
n_model.predict(input_text, verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


array([[1.]], dtype=float32)

In [ ]:
generate_text(n_model, tokenizer=tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=10)

'         '